In [27]:
import pandas as pd
import pandas as pd

# Data to be turned into a DataFrame
data = {
    "No bias":          [0.733, 0.525, 0.512, 0.421, 0.340, 0.334],
    "Self-diagnosis":   [0.700, 0.446, 0.253, 0.291, 0.172, 0.163],
    "Recency":          [0.685, 0.480, 0.310, 0.216, 0.227, 0.135],
    "Confirmation":     [0.717, 0.518, 0.386, 0.293, 0.309, ""],
    "Frequency":        [0.629, 0.384, 0.218, 0.179, "", ""],
    "Cultural":         [0.685, 0.428, 0.238, 0.226, "", ""],
    # "Blind spot": [0.453, 0.759, 0.422],
    "Status quo":       [0.684, 0.446, 0.318, 0.221, "", ""],
    "False consensus":  [0.637, 0.424, 0.208, 0.235, "", ""],
}

# Index for the DataFrame
index = [ "gpt-4-0613", "mixtral-8x7b-instruct-v0.1", "gpt-3.5-turbo-0613",
          "text-bison-001 (PaLM-2 text)", "pmc-llama-13b", "llama-2-70B-chat"]

# Creating the DataFrame
results_df = pd.DataFrame(data, index=index)

results_df

,No bias,Self-diagnosis,Recency,Confirmation,Frequency,Cultural,Status quo,False consensus
gpt-4-0613,0.733,0.700,0.685,0.717,0.629,0.685,0.684,0.637
mixtral-8x7b-instruct-v0.1,0.525,0.446,0.480,0.518,0.384,0.428,0.446,0.424
gpt-3.5-turbo-0613,0.512,0.253,0.310,0.386,0.218,0.238,0.318,0.208
text-bison-001 (PaLM-2 text),0.421,0.291,0.216,0.293,0.179,0.226,0.221,0.235
pmc-llama-13b,0.340,0.172,0.227,0.309,,,,
llama-2-70B-chat,0.334,0.163,0.135,,,,,


In [28]:
def query_results_df(results_df, model):
    curr_model = results_df.loc[model]

    cols = results_df.columns

    r = []
    theta = []
    for col in cols:
        if curr_model[col] != "":
            r.append(curr_model[col])
            theta.append(col)
    df = pd.DataFrame(dict(r=r, theta=theta))
    return df

In [32]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Define a set of distinct colors for the plots
colors = px.colors.qualitative.Set1

# Determine the number of plots/models
num_models = len(index)

# Specify the number of rows as an input variable
num_rows = 2  # Change this to your desired number of rows

# Calculate the number of columns needed based on the number of models and rows
num_cols = -(-num_models // num_rows)  # Ceiling division to ensure enough columns

# Create a subplot with specified number of rows and columns
fig = make_subplots(rows=num_rows, cols=num_cols,
                    specs=[[{'type': 'polar'}] * num_cols] * num_rows)

for i, model in enumerate(index):
    df = query_results_df(results_df, model)
    # Determine the row and column to place each subplot
    row = (i // num_cols) + 1
    col = (i % num_cols) + 1
    
    # Add each model's data to its own subplot with a unique color and label for the legend
    fig.add_trace(go.Scatterpolar(
        r=df['r'],
        theta=df['theta'],
        fill='toself',
        name=model,  # Assign the model name directly
        marker_color=colors[i],  # Assign a color from the predefined set
        # text=df['r'],  # Add text labels for each point
        mode='lines+markers+text',  # Display lines, markers, and text
        textposition='top center'  # Position the text above each marker
    ), row=row, col=col)

# Update layout for a more compact view and clear legend display
fig.update_layout(
    title="Model Comparison",
    legend_title="Models",
    legend=dict(
        orientation="h",  # Horizontal orientation
        yanchor="bottom",  # Anchor point is at the bottom
        y=-0.25,  # Position the legend below the plot
        xanchor="center",  # Center the legend
        x=0.5  # Position the legend in the middle
    ),
    height=300 * num_rows  # Adjust height based on the number of rows
)

# Update polar layout for each subplot if needed
for i in range(num_models):
    fig.update_layout(
        **{f'polar{i+1}': dict(
            radialaxis=dict(
                visible=True,
                range=[0, 1]
            )
        )}
    )

fig.show()


In [31]:
import plotly.graph_objects as go
import plotly.express as px

# Define a set of distinct colors for the plots
colors = px.colors.qualitative.Set1

# Create a single polar plot with a specified width
fig = go.Figure(layout=go.Layout(width=700))  # Adjust width as needed

for i, model in enumerate(index):
    df = query_results_df(results_df, model)
    # Add each model's data to the plot with a unique color and label for the legend
    fig.add_trace(go.Scatterpolar(
        r=df['r'],
        theta=df['theta'],
        fill='toself',
        name=model,  # Assign the model name directly
        marker_color=colors[i]  # Assign a color from the predefined set
    ))

# Update layout for a more compact view and clear legend display
fig.update_layout(
    title="Model Comparison",
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1]
        )
    ),
    legend_title="Models",
    legend=dict(
        orientation="h",  # Horizontal orientation
        yanchor="bottom",  # Anchor point is at the bottom
        y=-0.25,  # Position the legend below the plot
        xanchor="left",  # Anchor point is on the left
        x=0  # Position the legend at the far left
    )
)

fig.show()
